In [1]:
from scipy.stats import norm
import scipy.integrate as ing
import numpy as np
import numpy.random as rng
import scipy.stats as sts


In [2]:
def calculate_performance(M, n, alpha, theta, mu, sigma, tau):
    qstar = sts.gamma.ppf(tau,1.25,loc=0,scale=88)  #Qstar gamma distribution
    profit_qstar = calculate_profit(qstar, tau)
    profit_np_values = [] # store values profit
    profit_p_values = []

    for _ in range(M):
        data = rng.gamma(alpha, scale=theta, size=n)

        # parametric
        sample_mean = np.mean(data)
        sample_variance_unadj = np.var(data)
        qparam = norm.ppf(tau, sample_mean, np.sqrt(sample_variance_unadj))
        profit_param = calculate_profit(qparam, tau)# compute parametric profit
        profit_p_values.append(profit_qstar - profit_param )

        # non parametric
        qnonparam = get_order_statistic(data, int(np.ceil(tau * n)-1)) # to improve
        #add coomputing confidence interval
        profit_np = calculate_profit(qnonparam, tau)
        profit_np_values.append(profit_qstar - profit_np) #compute np profit

    plr_np = (np.sum(abs((profit_np_values)/profit_qstar))/ M)
    plr_p = (np.sum(abs((profit_p_values)/profit_qstar))/ M)
    plr_ratio = plr_np / plr_p
    return plr_ratio

def get_order_statistic(data, order):
    return np.partition(np.asarray(data), order)[order]

def int_normal(y):
    f = norm.pdf(y,loc=120,scale=15)
    return y * f

def calculate_profit(Q, tau):
    c = 1-tau
    p = 1
    t1 = p * ing.quad(int_normal,0,Q)[0]
    t2 = p * Q * norm.sf(Q,loc=120,scale=15)        # .sf is the survival function, i.e., 1-F(x)
    t3 = c * Q

    profit = t1 + t2 - t3
    return profit

In [3]:
def main():
    n = [10, 50, 100, 200]
    tau = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
    M = 1000
    alpha = 1.25
    theta = 88
    mu = 120
    sigma = 15

    for i in n:
        for j in tau:
            print(calculate_performance(M, i, alpha, theta, mu, sigma, j))

main()

0.13037203022763244
0.21869623895757626
0.4023453871789173
0.8234966265695481
1.2328798206825433
1.121582169894151
1.0743269686982524
1.1568810581875513
1.1630593778903229
0.023037636536479614
0.06666856133421944
0.1680767208712955
0.5845528654750686
0.5534728715518696
0.45464863443709486
1.1367231827526014
0.9630879997532115
0.7729221240825775
0.01264133572262951
0.04347495433944088
0.10953561010256221
0.4390819933899419
0.37539855987791027
0.27498138834153163
1.1458006583931866
0.8081589610250344
0.58047542007676
0.009144624450170252
0.03132559755884584
0.07767677132418774
0.3458025779004366
0.2493479325093364
0.16076818155744027
1.1267638984347514
0.6739881120455521
0.4297932446238672
